This notebook is used to parse crsp data that only includes stocks that are ordinary/common shares and listed on the NYSE, AMEX, or NASDAQ. Also, we set negative prices equal to 'NA' since a negative price represents the midpoint of the bid-ask spread and we only want closing price values.

# CRSP Data

In [70]:
import pandas as pd
from pathlib import Path  
import datetime
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')


In [40]:
data = pd.read_csv("crsp_1926_2020.csv")
data.date = pd.to_datetime(data.date)

FileNotFoundError: [Errno 2] File crsp_1926_2020.csv does not exist: 'crsp_1926_2020.csv'

In [41]:
remove_indexes = data[(data['SHRCD'] != 10) & (data['SHRCD'] != 11)].index
data.drop(remove_indexes, inplace=True)
remove_indexes = data[(data['EXCHCD'] != 1) & (data['EXCHCD'] != 3) & (data['EXCHCD'] != 3)].index
data.drop(remove_indexes, inplace=True)

data.loc[data["PRC"] < 0, "PRC"] = 0 
data.loc[data["RET"] == 'C', "RET"] = 0
data.loc[data["SHRCD"] == 10, "SHRCD"] = "Ordinary" 
data.loc[data["SHRCD"] == 11, "SHRCD"] = "Common" 
data.loc[data["EXCHCD"] == 1, "EXCHCD"] = "NYSE" 
data.loc[data["EXCHCD"] == 2, "EXCHCD"] = "AMEX" 
data.loc[data["EXCHCD"] == 3, "EXCHCD"] = "NASDAQ" 

In [42]:
data = data.dropna()
data = data.sort_values('date')
data = data.reset_index()
data = data.drop('index', axis=1)

In [43]:
lowest_date = datetime.datetime(2010, 1, 29)
remove_indexes = data[data['date'] < lowest_date].index
data.drop(remove_indexes, inplace=True)
data = data.reset_index()
data = data.drop('index', axis=1)

In [49]:
filepath = Path('monthly_stock.csv')  
data.to_csv(filepath)  

# FFM Data 

In [78]:
ffm = pd.read_excel("F-F_Research_Data_Factors.xlsx")

In [79]:
new_dates = []
for item in ffm.values:
    year = str(item[0])[:4]
    month = str(item[0])[4:]
    new_dates.append(year + "-" + month )

for index in range(len(ffm['Date'])):
    ffm['Date'][index] = new_dates[index]

market_returns = ffm['Mkt-RF'] + ffm['RF']
ffm['Mkt'] = market_returns

monthly_ffm = ffm.loc[:1159]
annual_ffm =  ffm.loc[1163:]

monthly_ffm.Date = pd.to_datetime(monthly_ffm.Date)

for index in range(len(monthly_ffm['Date'])):
    monthly_ffm['Date'][index] = monthly_ffm['Date'][index] + relativedelta(day=31)
#data['RET-Next'] = data.groupby('PERMNO')['RET'].shift(-1)
#data["RET-Next"].fillna(0, inplace=True)'''

In [80]:
lowest_date = datetime.datetime(2010, 1, 29)
remove_indexes = monthly_ffm[monthly_ffm['Date'] < lowest_date].index
monthly_ffm.drop(remove_indexes, inplace=True)
monthly_ffm = monthly_ffm.reset_index()
monthly_ffm = monthly_ffm.drop('index', axis=1)

In [87]:
filepath = Path('monthly_stock_ffm.csv')  
monthly_ffm.to_csv(filepath)  